In [1]:
import tensorflow as tf
from tensorflow import keras 
import torch
import numpy as np
import pandas as pd
import os
import pickle
from IPython.display import Image


In [8]:
# !git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [2]:
# %cd yolov5

In [11]:
# !pip install -r requirements.txt

  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0


In [2]:
%cd ml-chest-xray-annotator
%pwd
%ls

[WinError 2] The system cannot find the file specified: 'ml-chest-xray-annotator'
c:\Users\Samson\Documents\code\cp2\ml-chest-xray-annotator
 Volume in drive C is Windows
 Volume Serial Number is AE06-1B20

 Directory of c:\Users\Samson\Documents\code\cp2\ml-chest-xray-annotator

21/12/2022  12:07 AM    <DIR>          .
21/12/2022  12:07 AM    <DIR>          ..
20/12/2022  11:38 AM    <DIR>          .git
09/12/2022  01:08 PM               208 .gitignore
14/12/2022  09:25 AM    <DIR>          data
20/12/2022  11:21 AM            14,216 environment.yml
03/12/2022  02:25 AM    <DIR>          markdown
28/11/2022  09:16 PM    <DIR>          mine
13/12/2022  10:11 PM         2,766,936 model.ipynb
03/12/2022  04:01 AM    <DIR>          preprocessed
21/12/2022  02:58 AM    <DIR>          processed
27/10/2022  11:52 PM               137 README.md
20/12/2022  11:45 AM                 0 requirements.txt
19/12/2022  12:10 PM    <DIR>          virtualenv
21/12/2022  06:03 AM         7,823,787 yolo.

In [3]:
# train_dir = 'C:/Users/Samson/OneDrive - Strathmore University/ADS/ADS-Assignment-1'
train_dir = 'C:/Users/Samson/OneDrive - Strathmore University/dataset/train'
clean_data = pd.read_csv('C:/Users/Samson/Documents/code/cp2/ml-chest-xray-annotator/data/clean_data.csv')

In [4]:
clean_data.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,x_center,y_center,b_width,b_height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,10,691.0,1375.0,1653.0,1831.0,1172.0,1603.0,962.0,456.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,10,1264.0,743.0,1611.0,1019.0,1437.5,881.0,347.0,276.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#create Bounding boxes
from collections import namedtuple
Bounding_Box=[]
def buildBoundingBox(key):
    B = clean_data.loc[(clean_data['image_id']==key)]
    # print(data)
    boxes=[]
    count =0
    class_id = np.array(B["class_id"])
    rad_id = np.array(B["rad_id"])
    
    x_center = np.array(B['x_center'])
    y_center = np.array(B['y_center'])
    b_width = np.array(B['b_width'])
    b_height = np.array(B['b_height'])
    

    for i in range(len(x_center)):
        new_class_id=int(class_id[i])
        new_rad_id=int(rad_id[i])
        new_ycenter = int(x_center[i])
        new_xcenter = int(y_center[i])
        new_b_width = int(b_width[i])
        new_b_height = int(b_height[i])
        BoundingBox = str(new_class_id)+' '+str(new_xcenter)+' '+str(new_ycenter)+' '+str(new_b_width)+' '+str(new_b_height)

            # BoundingBox=(new_class_id,new_xmin,new_ymin,new_xmax,new_ymax)
        boxes.append(BoundingBox)
            # print(label(new_class_id,new_rad_id,new_xmin,new_ymin,new_xmax,new_ymax))
            # print(BoundingBox(new_xmin,new_ymin,new_xmax,new_ymax))
        
        return boxes
    count+=1
    print(boxes)
   


# buildBoundingBox("321c111713c3ee5385db0effb54ff568")


In [6]:
for x in os.scandir(train_dir):
    key =x.name[:-4]
    # key ="321c111713c3ee5385db0effb54ff568"
    B = buildBoundingBox(key) 

    f  =open(train_dir+'/'+key+'.txt','w')
    
    if B is not None:
        for b in B:
            f.write(b)
            f.write('\n')
        f.close()

    # pickle_out = open(train_dir+'/'+x.name+'.txt','w')
    # pickle.dump(B,pickle_out)
    # pickle_out.close()
    
print(B)

['14 0 0 0 0']


#Divide The Data into Train and Test

In [7]:
import os 
from random import choice
import shutil

images =[]
labels=[]

train_path ='C:/Users/Samson/OneDrive - Strathmore University/dataset/processed/images/train/'
test_path ='C:/Users/Samson/OneDrive - Strathmore University/dataset/processed/images/val/'
crs_path = 'C:/Users/Samson/OneDrive - Strathmore University/dataset/train/'

train_ratio= 0.8
test_ratio =0.2

total_image_count = len(os.listdir(crs_path))/2

for (dirname,dirs,files) in os.walk(crs_path):
    for filename in files:
        if filename.endswith('.txt'):
            labels.append(filename)
        else:
            images.append(filename)


train_count = int(len(images)*train_ratio)
test_count = int(len(images)*test_ratio)
print("training images are:",train_count)
print("testing Images are:",test_count)



training images are: 12000
testing Images are: 3000


In [8]:
trainImagePath ="C:/Users/Samson/OneDrive - Strathmore University/dataset/processed/images/train/"
trainLabelPath ="C:/Users/Samson/OneDrive - Strathmore University/dataset/processed/labels/train/"
testImagePath = "C:/Users/Samson/OneDrive - Strathmore University/dataset/processed/images/val/"
testLabelPath = "C:/Users/Samson/OneDrive - Strathmore University/dataset/processed/labels/val/"
#loop to create train data
for x in range (train_count):
    filePng = choice(images)#get name from orgin dir
    fileLable = filePng[:-4]+'.txt' #get name of corresponding file

    shutil.copy(os.path.join(crs_path,filePng),os.path.join(trainImagePath,filePng))
    shutil.copy(os.path.join(crs_path,fileLable),os.path.join(trainLabelPath,fileLable))

    images.remove(filePng)
    labels.remove(fileLable)


#loop to create test data
for x in range(test_count):
    filePng = choice(images)
    fileLable = filePng[:-4]+'.txt'

    #move both labels and image inot the test dir
    shutil.copy(os.path.join(crs_path,filePng),os.path.join(testImagePath,filePng))
    shutil.copy(os.path.join(crs_path,fileLable),os.path.join(testLabelPath,fileLable))
    
    #empty the lists 
    images.remove(filePng)
    labels.remove(fileLable)

#all files have been moved, rename original file path 
shutil.move(crs_path,test_path)



'C:/Users/Samson/OneDrive - Strathmore University/dataset/processed/images/val/train'

In [10]:
%cd yolov5
%pwd 

c:\Users\Samson\Documents\code\cp2\ml-chest-xray-annotator\yolov5


'c:\\Users\\Samson\\Documents\\code\\cp2\\ml-chest-xray-annotator\\yolov5'

In [13]:
!python train.py --img 415 --batch 8 --epochs 3 --data dataset.yaml --weights yolov5s.pt --cfg models/yolov5m.yaml --cache

train: weights=yolov5s.pt, cfg=models/yolov5m.yaml, data=dataset.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=3, batch_size=8, imgsz=415, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
From https://github.com/ultralytics/yolov5
   f72f0fe..2370a55  master     -> origin/master
   49abfe1..c67dc5d  exp13-soft -> origin/exp13-soft
github:  YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5  v7.0-43-gf72f0fe Python-3.8.8 torch-1.13.1 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)

hyperparameters: lr0=0.01, lrf

train: WARNING  C:\Users\Samson\OneDrive - Strathmore University\dataset\processed\images\train\02de1ec3340e75fd5e7294a84a0342a1.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [       1197        1336         482         474]
train: WARNING  C:\Users\Samson\OneDrive - Strathmore University\dataset\processed\images\train\02fe3eea41166a1d3eb75307a12c4a63.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        801        2089         115         119]
train: WARNING  C:\Users\Samson\OneDrive - Strathmore University\dataset\processed\images\train\033dae57cec0aca171d47090f299bed2.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [       1669        1103         949         330]
train: WARNING  C:\Users\Samson\OneDrive - Strathmore University\dataset\processed\images\train\033e1637bed9b9f3dccac9c6c419adc2.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        597      